In [6]:
import sys, os
sys.path.append(os.path.abspath('../..'))

In [7]:
from tqdm.notebook import tqdm
import math
import gym
import torch
import torch.optim as optim 
from torch.utils.tensorboard import SummaryWriter
from collections import deque

from networks.dqn_atari import DQN
from utils.memory import ReplayMemory
from utils.optimization import standard_optimization
from environments.atari_wrappers import make_atari, wrap_deepmind
from utils.atari_utils import fp, ActionSelector, evaluate

ValueError: attempted relative import beyond top-level package

In [5]:
env_name = 'Breakout'
env_raw = make_atari('{}NoFrameskip-v4'.format(env_name))
env = wrap_deepmind(env_raw, frame_stack=False, episode_life=True, clip_rewards=True)
c,h,w = c,h,w = fp(env.reset()).shape
n_actions = env.action_space.n

NameError: name 'make_atari' is not defined

In [ ]:
BATCH_SIZE = 64
LR = 0.0000625
GAMMA = 0.99
EPS_START = 1.
EPS_END = 0.1
EPS_DECAY = 1000000 
NUM_STEPS = 20000000
POLICY_UPDATE = 4
TARGET_UPDATE = 4000
EVALUATE_FREQ = 4000
MEMORY_CAPACITY = 100000
INITIAL_STEPS=1000
PERCENTAGE = 0.5

NAME = 'Standard_RL_original'

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # if gpu is to be used
policy_net = DQN(n_actions).to(device)
target_net = DQN(n_actions).to(device)
policy_net.apply(policy_net.init_weights)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()
optimizer = optim.Adam(policy_net.parameters(), lr=LR, eps=1.5e-4)

In [ ]:
memory = ReplayMemory(MEMORY_CAPACITY, [5,h,w], n_actions, device)
action_selector = ActionSelector(EPS_START, EPS_END, policy_net, EPS_DECAY, n_actions, device)

In [ ]:
steps_done = 0
writer = SummaryWriter()

In [ ]:
q = deque(maxlen=5)
done=True
eps = 0
episode_len = 0

In [ ]:
progressive = tqdm(range(NUM_STEPS), total=NUM_STEPS, ncols=400, leave=False, unit='b')
for step in progressive:
  if done:
    env.reset()
    sum_reward = 0
    episode_len = 0
    img, _, _, _ = env.step(1) # BREAKOUT specific !!!
    for i in range(10): # no-op
      n_frame, _, _, _ = env.step(0)
      n_frame = fp(n_frame)
      q.append(n_frame)
        
  # Select and perform an action
  state = torch.cat(list(q))[1:].unsqueeze(0)
  action, eps = action_selector.select_action(state)
  n_frame, reward, done, info = env.step(action)
  n_frame = fp(n_frame)

  # 5 frame as memory
  q.append(n_frame)
  memory.push(torch.cat(list(q)).unsqueeze(0), action, reward, done) # here the n_frame means next frame from the previous time step
  episode_len += 1

  # Perform one step of the optimization (on the target network)
  if step % POLICY_UPDATE == 0 and step > INITIAL_STEPS:
    loss = standard_optimization(policy_net, target_net, optimizer, memory, batch_size=BATCH_SIZE)
    if loss is not None:
      writer.add_scalar('Performance/loss', loss, step)
    
  # Update the target network, copying all weights and biases in DQN
  if step % TARGET_UPDATE == 0 and step > INITIAL_STEPS:
    target_net.load_state_dict(policy_net.state_dict())
    
  if step % EVALUATE_FREQ == 0 and step > INITIAL_STEPS:
    evaluated_reward = evaluate(step, policy_net, device, env_raw, n_actions, eps=0.05, num_episode=15)
    writer.add_scalar('Performance/reward', evaluated_reward, step)